# 🌳 Giải thích chi tiết thuật toán Decision Tree Classifier (Python)

> Đây là phần chú thích chi tiết từng đoạn code xây dựng `DecisionTreeClassifier` từ scratch bằng Python, dành cho người mới học Machine Learning.

## Import thư viện

In [1]:
import numpy as np
from collections import Counter

- `numpy` được dùng để xử lý mảng dữ liệu số học.
- `Counter` là một class từ `collections` giúp đếm tần suất xuất hiện của từng phần tử trong một danh sách.

## Lớp Node

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature          # Chỉ số thuộc tính
        self.threshold = threshold      # Ngưỡng chia
        self.left = left                # Nhánh trái
        self.right = right              # Nhánh phải
        self.value = value              # Nhãn nếu là node lá

    def is_leaf_node(self):
        return self.value is not None

- Mỗi `Node` biểu diễn một nút trong cây quyết định.
- Nếu là node lá (leaf node), nó chỉ lưu `value` là nhãn dự đoán.
- Nếu là node quyết định (decision node), nó lưu thông tin chia: `feature`, `threshold`, `left`, `right`.

## Lớp DecisionTreeClassifier

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None


- Đây là class chính để xây dựng cây phân loại.
- Có 2 siêu tham số (`hyperparameters`) quan trọng:
  - `max_depth`: kiểm soát overfitting bằng cách giới hạn độ sâu.
  - `min_samples_split`: nếu số mẫu nhỏ hơn ngưỡng này thì không chia nữa.

## Hàm fit và grow tree

In [ ]:
    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(set(y))

        # Điều kiện dừng
        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Tìm chia tốt nhất
        best_feat, best_thresh = self._best_split(X, y, num_features)
        if best_feat is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Chia tập
        left_idxs = X[:, best_feat] <= best_thresh
        right_idxs = X[:, best_feat] > best_thresh

        left = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)

        return Node(best_feat, best_thresh, left, right)


- Hàm `fit` bắt đầu huấn luyện cây bằng cách gọi `_grow_tree`.
- `_grow_tree` là hàm đệ quy để xây dựng cây từ dữ liệu huấn luyện.
- Dừng khi:
  - Đạt độ sâu tối đa.
  - Tập dữ liệu chỉ còn 1 nhãn.
  - Quá ít mẫu để chia tiếp.

## Tìm chia tốt nhất

In [ ]:
    def _best_split(self, X, y, num_features):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat_idx in range(num_features):
            thresholds = np.unique(X[:, feat_idx])
            for threshold in thresholds:
                left_idxs = y[X[:, feat_idx] <= threshold]
                right_idxs = y[X[:, feat_idx] > threshold]

                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                gain = self._gini_gain(y, left_idxs, right_idxs)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

- Với mỗi thuộc tính, thử tất cả các ngưỡng có thể chia.
- Đánh giá mỗi lần chia bằng chỉ số Gini gain.
- Lưu lại chia tốt nhất nếu gain lớn hơn.

## Tính Gini và Gini Gain

In [ ]:
    def _gini_gain(self, parent, left, right):
        weight_left = len(left) / len(parent)
        weight_right = len(right) / len(parent)
        return self._gini(parent) - (weight_left * self._gini(left) + weight_right * self._gini(right))

    def _gini(self, y):
        counts = np.bincount(y)
        probs = counts / len(y)
        return 1 - np.sum(probs**2)


- `Gini` là chỉ số đo độ hỗn tạp của một tập dữ liệu.
- `Gini Gain` đo mức độ giảm hỗn tạp sau khi chia dữ liệu.

## Lấy nhãn phổ biến nhất

In [ ]:
    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

- Dùng khi cần gán nhãn cho node lá.
- Lấy nhãn xuất hiện nhiều nhất trong tập y.

## Hàm dự đoán

In [ ]:
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

- Duyệt cây từ gốc đến lá để dự đoán nhãn đầu ra cho mỗi mẫu.

#Code Full

In [2]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature          # Chỉ số thuộc tính
        self.threshold = threshold      # Ngưỡng chia
        self.left = left                # Nhánh trái
        self.right = right              # Nhánh phải
        self.value = value              # Nhãn nếu là node lá

    def is_leaf_node(self):
        return self.value is not None

class DecisionTreeClassifier:
    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(set(y))

        # Điều kiện dừng
        if (depth >= self.max_depth or num_labels == 1 or num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Tìm chia tốt nhất
        best_feat, best_thresh = self._best_split(X, y, num_features)
        if best_feat is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # Chia tập
        left_idxs = X[:, best_feat] <= best_thresh
        right_idxs = X[:, best_feat] > best_thresh

        left = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)

        return Node(best_feat, best_thresh, left, right)

    def _best_split(self, X, y, num_features):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat_idx in range(num_features):
            thresholds = np.unique(X[:, feat_idx])
            for threshold in thresholds:
                left_idxs = y[X[:, feat_idx] <= threshold]
                right_idxs = y[X[:, feat_idx] > threshold]

                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                gain = self._gini_gain(y, left_idxs, right_idxs)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _gini_gain(self, parent, left, right):
        weight_left = len(left) / len(parent)
        weight_right = len(right) / len(parent)
        return self._gini(parent) - (weight_left * self._gini(left) + weight_right * self._gini(right))

    def _gini(self, y):
        counts = np.bincount(y)
        probs = counts / len(y)
        return 1 - np.sum(probs**2)

    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


## Đoạn mã Test Code

In [3]:
# ===========================================
# Test thử bộ code
# ===========================================

# Tạo bộ dữ liệu giả lập (2 đặc trưng và 2 lớp phân loại)
X = np.array([[2, 3],
              [3, 4],
              [1, 1],
              [6, 7],
              [7, 8],
              [5, 6]])

y = np.array([0, 0, 0, 1, 1, 1])

# Khởi tạo và huấn luyện mô hình Decision Tree
model = DecisionTreeClassifier(max_depth=3, min_samples_split=2)
model.fit(X, y)

# Dự đoán cho các mẫu trong X
y_pred = model.predict(X)

# In kết quả dự đoán
print("Dự đoán:", y_pred)

# In cấu trúc cây (để kiểm tra cây đã được xây dựng đúng hay không)
def print_tree(node, depth=0):
    if node.is_leaf_node():
        print("II" * depth + f"Leaf: {node.value}")
        return
    print("  " * depth + f"Feature: {node.feature}, Threshold: {node.threshold}")
    print_tree(node.left, depth + 1)
    print_tree(node.right, depth + 1)

# In cây quyết định đã học được
print("Cấu trúc cây quyết định:")
print_tree(model.root)


Dự đoán: [0 0 0 1 1 1]
Cấu trúc cây quyết định:
Feature: 0, Threshold: 3
IILeaf: 0
IILeaf: 1
